In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
pd.set_option('display.max_columns', None)
data = pd.read_csv('../data/final_data_corrected.csv')
data.head(1)

,Date,Time,TeamH,TeamA,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,HStr,HSOH,HSOA,HSAH,HSAA,HSDH,HSDA,HO,HA,HM,HD,HDP,HIP,HSAAg,HATAAg,HGls,HAssts,HGlsPK,HxG,HNpxG,HxAG,HPoss,HPrg,HPrgP,HGKS%,HGKCS%,HSoT,Hg-xG,HCmp,HCmp%,HPrgDist,HXa,HKP,HPPA,HCrsPA,HTB,HCrs,HCK,HSCA,HGCA,HTkIW,HLost,HBlocks,HPass,HInt,HClr,HAttPen,HSucc%,HCPA,HDis,HPrgR,HOnG,HOnGA,HOnxG,HOnxGA,AStr,ASOH,ASOA,ASAH,ASAA,ASDH,ASDA,AO,AA,AM,AD,ADP,AIP,ASAAg,ATAAg,AGls,AAssts,AGlsPK,AxG,ANpxG,AxAG,APoss,APrg,APrgP,AGKS%,AGKCS%,ASoT,Ag-xG,ACmp,ACmp%,APrgDist,AXa,AKP,APPA,ACrsPA,ATB,ACrs,ACK,ASCA,AGCA,ATkIW,ALost,ABlocks,APass,AInt,AClr,AAttPen,ASucc%,ACPA,ADis,APrgR,AOnG,AOnGA,AOnxG,AOnxGA
0,5/8/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,4.2,3.6,1.85,4.33,3.5,1.87,4.3,3.55,1.85,4.5,3.65,1.89,4.4,3.5,1.83,4.6,3.5,1.87,4.6,3.78,1.95,4.39,3.59,1.88,2.1,1.72,2.14,1.78,2.19,1.91,2.09,1.76,0.5,2.04,1.89,2.03,1.89,2.06,1.91,2.01,1.87,4.5,3.6,1.8,4.5,3.5,1.83,4.4,3.55,1.85,4.58,3.63,1.88,4.8,3.4,1.78,4.75,3.5,1.85,5.01,3.7,1.91,4.56,3.57,1.85,2.1,1.72,2.14,1.78,2.19,1.91,2.08,1.76,0.5,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85,3,1085,1100,1110,1110,1060,1090,76,77,75,76,4,2,26.6,25.2,1.0,0.76,0.97,1.02,0.96,0.75,45.8,14.2,32.2,71.8,24.3,3.51,-0.02,348.8,77.2,2310.4,0.61,8.19,6.14,1.27,0.95,15.4,4.89,19.4,1.76,11.2,6.73,12.3,9.08,9.65,21.5,19.9,46.9,5.41,12.1,32.1,1.05,1.3,1.02,1.29,4,1245,1285,1250,1250,1240,1320,80,83,81,79,7,7,24.1,23.0,2.14,1.62,2.05,1.87,1.79,1.39,59.5,21.7,54.2,70.6,35.1,5.03,0.27,482.8,83.1,2658.9,1.21,11.7,12.01,1.65,2.24,17.8,5.81,27.6,3.81,8.89,6.51,9.49,7.22,6.35,15.8,34.0,46.3,7.38,10.1,53.5,2.24,1.16,1.87,1.12


In [3]:
X = data[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
          'PSH', 'PSD', 'PSA', 'WHH','WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH', 'AvgD', 'AvgA', 'B365>2.5',
          'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH','B365AHA', 'PAHH', 'PAHA', 
          'MaxAHH', 'MaxAHA', "AvgAHH", 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA',
          'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD',
          'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH',
          'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HDP', 'HIP', 'HSAAg', 'HATAAg', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG',
          'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA',
          'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA',
          'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA',
          'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP',
          'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK',
          'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr','AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG',
          'AOnGA', 'AOnxG', 'AOnxGA']].copy()
y = data['FTHG']

In [4]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [6]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 1.98004028
Iteration 2, loss = 1.11124240
Iteration 3, loss = 0.97281161
Iteration 4, loss = 1.06338656
Iteration 5, loss = 1.06363056
Iteration 6, loss = 0.93531796
Iteration 7, loss = 0.81966051
Iteration 8, loss = 0.76585786
Iteration 9, loss = 0.78774715
Iteration 10, loss = 0.80982198
Iteration 11, loss = 0.80110747
Iteration 12, loss = 0.76320966
Iteration 13, loss = 0.73852425
Iteration 14, loss = 0.73208343
Iteration 15, loss = 0.73070140
Iteration 16, loss = 0.73028570
Iteration 17, loss = 0.72110242
Iteration 18, loss = 0.70548456
Iteration 19, loss = 0.69362826
Iteration 20, loss = 0.68982115
Iteration 21, loss = 0.68772707
Iteration 22, loss = 0.68613139
Iteration 23, loss = 0.68065969
Iteration 24, loss = 0.66983870
Iteration 25, loss = 0.66284851
Iteration 26, loss = 0.65784909
Iteration 27, loss = 0.65312928
Iteration 28, loss = 0.64815640
Iteration 29, loss = 0.64240157
Iteration 30, loss = 0.63813379
Iteration 31, loss = 0.63377182
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [7]:
print(f'R2 Score: {r2_score(y_test, pipe.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2 Score: -0.13981115494293372
MAE: 1.1758892041076554


In [8]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 1.67822909
Iteration 2, loss = 1.09522649
Iteration 3, loss = 0.92181051
Iteration 4, loss = 0.93536560
Iteration 5, loss = 0.86203706
Iteration 6, loss = 0.78757793
Iteration 7, loss = 0.79062191
Iteration 8, loss = 0.80299741
Iteration 9, loss = 0.77886143
Iteration 10, loss = 0.75442367
Iteration 11, loss = 0.74166664
Iteration 12, loss = 0.74546890
Iteration 13, loss = 0.73094521
Iteration 14, loss = 0.70999464
Iteration 15, loss = 0.70674768
Iteration 16, loss = 0.70500956
Iteration 17, loss = 0.69588432
Iteration 18, loss = 0.68169086
Iteration 19, loss = 0.66880467
Iteration 20, loss = 0.66256863
Iteration 21, loss = 0.65329629
Iteration 22, loss = 0.64320215
Iteration 23, loss = 0.63317083
Iteration 24, loss = 0.62381105
Iteration 25, loss = 0.61510103
Iteration 26, loss = 0.60415051
Iteration 27, loss = 0.59726698
Iteration 28, loss = 0.58589941
Iteration 29, loss = 0.57088942
Iteration 30, loss = 0.55762545
Iteration 31, loss = 0.54896504
Iteration 32, los

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [9]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.34148874755320646
MAE: 1.3211367795319207


In [10]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (60, 60, 60),
                            alpha = 24))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 20.65804564
Iteration 2, loss = 19.57560607
Iteration 3, loss = 18.85015981
Iteration 4, loss = 18.24848740
Iteration 5, loss = 17.71915139
Iteration 6, loss = 17.25022833
Iteration 7, loss = 16.80063355
Iteration 8, loss = 16.34260600
Iteration 9, loss = 15.87503561
Iteration 10, loss = 15.41024118
Iteration 11, loss = 14.96686071
Iteration 12, loss = 14.53812603
Iteration 13, loss = 14.12730823
Iteration 14, loss = 13.72438752
Iteration 15, loss = 13.32875623
Iteration 16, loss = 12.94455655
Iteration 17, loss = 12.56838420
Iteration 18, loss = 12.20567357
Iteration 19, loss = 11.85613556
Iteration 20, loss = 11.51333945
Iteration 21, loss = 11.18304733
Iteration 22, loss = 10.86122412
Iteration 23, loss = 10.55012986
Iteration 24, loss = 10.24768261
Iteration 25, loss = 9.95515491
Iteration 26, loss = 9.67037918
Iteration 27, loss = 9.39369923
Iteration 28, loss = 9.12616454
Iteration 29, loss = 8.86596097
Iteration 30, loss = 8.61341500
Iteration 31, loss = 8.36

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=24, hidden_layer_sizes=(60, 60, 60),
                              verbose=True))])

In [11]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: 0.12581259062671335
MAE: 1.071949933640317


In [12]:
X = data[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
          'PSH', 'PSD', 'PSA', 'WHH','WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH', 'AvgD', 'AvgA', 'B365>2.5',
          'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH','B365AHA', 'PAHH', 'PAHA', 
          'MaxAHH', 'MaxAHA', "AvgAHH", 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA',
          'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD',
          'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH',
          'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HDP', 'HIP', 'HSAAg', 'HATAAg', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG',
          'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA',
          'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA',
          'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA',
          'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP',
          'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK',
          'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr','AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG',
          'AOnGA', 'AOnxG', 'AOnxGA']].copy()
y = data['FTAG']

In [13]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [15]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 1.75726645
Iteration 2, loss = 0.92216443
Iteration 3, loss = 0.82591465
Iteration 4, loss = 0.91220096
Iteration 5, loss = 0.85336783
Iteration 6, loss = 0.71470989
Iteration 7, loss = 0.63457748
Iteration 8, loss = 0.61772820
Iteration 9, loss = 0.63383465
Iteration 10, loss = 0.63362715
Iteration 11, loss = 0.60863185
Iteration 12, loss = 0.58334355
Iteration 13, loss = 0.58087194
Iteration 14, loss = 0.57896569
Iteration 15, loss = 0.56947465
Iteration 16, loss = 0.55249908
Iteration 17, loss = 0.54367786
Iteration 18, loss = 0.53906529
Iteration 19, loss = 0.53607321
Iteration 20, loss = 0.52857498
Iteration 21, loss = 0.51997508
Iteration 22, loss = 0.51205291
Iteration 23, loss = 0.50760139
Iteration 24, loss = 0.50284651
Iteration 25, loss = 0.49681073
Iteration 26, loss = 0.48981677
Iteration 27, loss = 0.48318649
Iteration 28, loss = 0.48081802
Iteration 29, loss = 0.47683734
Iteration 30, loss = 0.47037670
Iteration 31, loss = 0.46431301
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [16]:
print(f'R2 Score: {r2_score(y_test, pipe.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2 Score: -0.305002591156297
MAE: 1.026986455526792


In [17]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 2.10429423
Iteration 2, loss = 1.21257260
Iteration 3, loss = 0.79931530
Iteration 4, loss = 0.73813855
Iteration 5, loss = 0.79268255
Iteration 6, loss = 0.73225883
Iteration 7, loss = 0.65225055
Iteration 8, loss = 0.63750100
Iteration 9, loss = 0.64005598
Iteration 10, loss = 0.62661837
Iteration 11, loss = 0.59886054
Iteration 12, loss = 0.58773123
Iteration 13, loss = 0.59825353
Iteration 14, loss = 0.59114382
Iteration 15, loss = 0.57467706
Iteration 16, loss = 0.56374549
Iteration 17, loss = 0.56062151
Iteration 18, loss = 0.55359324
Iteration 19, loss = 0.54488873
Iteration 20, loss = 0.53746903
Iteration 21, loss = 0.52923770
Iteration 22, loss = 0.52366433
Iteration 23, loss = 0.51883685
Iteration 24, loss = 0.50939166
Iteration 25, loss = 0.50145740
Iteration 26, loss = 0.49368709
Iteration 27, loss = 0.48776025
Iteration 28, loss = 0.48207020
Iteration 29, loss = 0.47263465
Iteration 30, loss = 0.46403628
Iteration 31, loss = 0.45634121
Iteration 32, los

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [18]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.46907883083724067
MAE: 1.120587810676176


In [19]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (60, 60, 60),
                            alpha = 24))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 19.21687066
Iteration 2, loss = 18.50375430
Iteration 3, loss = 17.91587672
Iteration 4, loss = 17.38923182
Iteration 5, loss = 16.88084904
Iteration 6, loss = 16.37438867
Iteration 7, loss = 15.87090595
Iteration 8, loss = 15.38536108
Iteration 9, loss = 14.91522662
Iteration 10, loss = 14.46184335
Iteration 11, loss = 14.01745271
Iteration 12, loss = 13.58286327
Iteration 13, loss = 13.16111900
Iteration 14, loss = 12.74906159
Iteration 15, loss = 12.35201278
Iteration 16, loss = 11.96712546
Iteration 17, loss = 11.59416915
Iteration 18, loss = 11.23236800
Iteration 19, loss = 10.88142061
Iteration 20, loss = 10.54185873
Iteration 21, loss = 10.21367413
Iteration 22, loss = 9.89637729
Iteration 23, loss = 9.58852227
Iteration 24, loss = 9.28963239
Iteration 25, loss = 9.00113810
Iteration 26, loss = 8.72190060
Iteration 27, loss = 8.45257500
Iteration 28, loss = 8.19203694
Iteration 29, loss = 7.93902835
Iteration 30, loss = 7.69575884
Iteration 31, loss = 7.46006

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 75, loss = 2.19138315
Iteration 76, loss = 2.14126217
Iteration 77, loss = 2.09293646
Iteration 78, loss = 2.04601709
Iteration 79, loss = 2.00134811
Iteration 80, loss = 1.95771422
Iteration 81, loss = 1.91562493
Iteration 82, loss = 1.87499137
Iteration 83, loss = 1.83573150
Iteration 84, loss = 1.79785760
Iteration 85, loss = 1.76142719
Iteration 86, loss = 1.72554768
Iteration 87, loss = 1.69155768
Iteration 88, loss = 1.65871548
Iteration 89, loss = 1.62799382
Iteration 90, loss = 1.59811297
Iteration 91, loss = 1.56752514
Iteration 92, loss = 1.53808223
Iteration 93, loss = 1.50981242
Iteration 94, loss = 1.48345928
Iteration 95, loss = 1.45739210
Iteration 96, loss = 1.43214607
Iteration 97, loss = 1.40773012
Iteration 98, loss = 1.38473152
Iteration 99, loss = 1.36221851
Iteration 100, loss = 1.34037590
Iteration 101, loss = 1.31919073
Iteration 102, loss = 1.29940224
Iteration 103, loss = 1.27917875
Iteration 104, loss = 1.26040455
Iteration 105, loss = 1.24296140
It

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=24, hidden_layer_sizes=(60, 60, 60),
                              verbose=True))])

In [20]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: 0.02130403775301326
MAE: 0.9324010339023746


In [21]:
pd.set_option('display.max_columns', None)
data1 = pd.read_csv('../data/data_betting.csv')
data1.head(1)

,Unnamed: 0,Date,Time,TeamH,TeamA,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,0,5/8/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,0,0,4.2,3.6,1.85,4.33,3.5,1.87,4.3,3.55,1.85,4.5,3.65,1.89,4.4,3.5,1.83,4.6,3.5,1.87,4.6,3.78,1.95,4.39,3.59,1.88,2.1,1.72,2.14,1.78,2.19,1.91,2.09,1.76,0.5,2.04,1.89,2.03,1.89,2.06,1.91,2.01,1.87,4.5,3.6,1.8,4.5,3.5,1.83,4.4,3.55,1.85,4.58,3.63,1.88,4.8,3.4,1.78,4.75,3.5,1.85,5.01,3.7,1.91,4.56,3.57,1.85,2.1,1.72,2.14,1.78,2.19,1.91,2.08,1.76,0.5,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85


In [22]:
X = data1[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
          'PSH', 'PSD', 'PSA', 'WHH','WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH', 'AvgD', 'AvgA', 'B365>2.5',
          'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH','B365AHA', 'PAHH', 'PAHA', 
          'MaxAHH', 'MaxAHA', "AvgAHH", 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA',
          'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD',
          'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH',
          'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA']].copy()
y = data1['FTHG'] 

In [23]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [25]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 2.28047432
Iteration 2, loss = 1.79498469
Iteration 3, loss = 1.41802557
Iteration 4, loss = 1.15541720
Iteration 5, loss = 1.00862811
Iteration 6, loss = 0.94517856
Iteration 7, loss = 0.92446690
Iteration 8, loss = 0.92349852
Iteration 9, loss = 0.91480142
Iteration 10, loss = 0.88743949
Iteration 11, loss = 0.84826341
Iteration 12, loss = 0.80514833
Iteration 13, loss = 0.76974018
Iteration 14, loss = 0.74397858
Iteration 15, loss = 0.73229210
Iteration 16, loss = 0.72631441
Iteration 17, loss = 0.72260514
Iteration 18, loss = 0.72099465
Iteration 19, loss = 0.71458452
Iteration 20, loss = 0.70598872
Iteration 21, loss = 0.69594851
Iteration 22, loss = 0.68497333
Iteration 23, loss = 0.67633900
Iteration 24, loss = 0.67073525
Iteration 25, loss = 0.67122793
Iteration 26, loss = 0.66756912
Iteration 27, loss = 0.66318330
Iteration 28, loss = 0.65740152
Iteration 29, loss = 0.64898692
Iteration 30, loss = 0.64253652
Iteration 31, loss = 0.63718371
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [26]:
print(f'R2 Score: {r2_score(y_test, pipe.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2 Score: -0.227685898983224
MAE: 1.2377881120432657


In [27]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 1.52423785
Iteration 2, loss = 1.15110207
Iteration 3, loss = 0.95398767
Iteration 4, loss = 0.93111138
Iteration 5, loss = 0.92864420
Iteration 6, loss = 0.86031377
Iteration 7, loss = 0.79178200
Iteration 8, loss = 0.77058783
Iteration 9, loss = 0.75275986
Iteration 10, loss = 0.72903756
Iteration 11, loss = 0.70982554
Iteration 12, loss = 0.69684197
Iteration 13, loss = 0.69274512
Iteration 14, loss = 0.67828185
Iteration 15, loss = 0.66388523
Iteration 16, loss = 0.64897201
Iteration 17, loss = 0.63928987
Iteration 18, loss = 0.62607170
Iteration 19, loss = 0.61592085
Iteration 20, loss = 0.60747095
Iteration 21, loss = 0.59699611
Iteration 22, loss = 0.58213126
Iteration 23, loss = 0.56629898
Iteration 24, loss = 0.55419378
Iteration 25, loss = 0.54088288
Iteration 26, loss = 0.52663338
Iteration 27, loss = 0.51424944
Iteration 28, loss = 0.49831422
Iteration 29, loss = 0.48436755
Iteration 30, loss = 0.46919552
Iteration 31, loss = 0.45511619
Iteration 32, los

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [28]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.3161666383751398
MAE: 1.2690083210638925


In [29]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (60, 60, 60),
                            alpha = 24))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 19.94748872
Iteration 2, loss = 19.31312084
Iteration 3, loss = 18.71355895
Iteration 4, loss = 18.14558696
Iteration 5, loss = 17.58921941
Iteration 6, loss = 17.05918693
Iteration 7, loss = 16.54000436
Iteration 8, loss = 16.02950133
Iteration 9, loss = 15.53275018
Iteration 10, loss = 15.05170980
Iteration 11, loss = 14.57650305
Iteration 12, loss = 14.13891063
Iteration 13, loss = 13.70768626
Iteration 14, loss = 13.30967653
Iteration 15, loss = 12.94500946
Iteration 16, loss = 12.59296951
Iteration 17, loss = 12.25966202
Iteration 18, loss = 11.93393348
Iteration 19, loss = 11.61391823
Iteration 20, loss = 11.29664164
Iteration 21, loss = 10.98548592
Iteration 22, loss = 10.68204887
Iteration 23, loss = 10.39000049
Iteration 24, loss = 10.10998511
Iteration 25, loss = 9.83776520
Iteration 26, loss = 9.57417808
Iteration 27, loss = 9.31811405
Iteration 28, loss = 9.06860673
Iteration 29, loss = 8.82595613
Iteration 30, loss = 8.58844175
Iteration 31, loss = 8.35

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 69, loss = 3.21133641
Iteration 70, loss = 3.14008557
Iteration 71, loss = 3.07101735
Iteration 72, loss = 3.00445982
Iteration 73, loss = 2.93933260
Iteration 74, loss = 2.87645280
Iteration 75, loss = 2.81584812
Iteration 76, loss = 2.75700380
Iteration 77, loss = 2.69940893
Iteration 78, loss = 2.64413871
Iteration 79, loss = 2.59127229
Iteration 80, loss = 2.53918886
Iteration 81, loss = 2.48938828
Iteration 82, loss = 2.43979885
Iteration 83, loss = 2.39302498
Iteration 84, loss = 2.34735891
Iteration 85, loss = 2.30253792
Iteration 86, loss = 2.25983115
Iteration 87, loss = 2.21841201
Iteration 88, loss = 2.17945267
Iteration 89, loss = 2.14070150
Iteration 90, loss = 2.10207528
Iteration 91, loss = 2.06515235
Iteration 92, loss = 2.02978753
Iteration 93, loss = 1.99635146
Iteration 94, loss = 1.96359180
Iteration 95, loss = 1.93257537
Iteration 96, loss = 1.90103918
Iteration 97, loss = 1.86985224
Iteration 98, loss = 1.84132400
Iteration 99, loss = 1.81416977
Iteratio

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=24, hidden_layer_sizes=(60, 60, 60),
                              verbose=True))])

In [30]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: 0.058781472486814756
MAE: 1.1288374913776031


In [31]:
X = data1[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
          'PSH', 'PSD', 'PSA', 'WHH','WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH', 'AvgD', 'AvgA', 'B365>2.5',
          'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH','B365AHA', 'PAHH', 'PAHA', 
          'MaxAHH', 'MaxAHA', "AvgAHH", 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA',
          'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD',
          'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH',
          'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA']].copy()
y = data1['FTAG'] 

In [32]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [34]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 1.80956812
Iteration 2, loss = 1.39248068
Iteration 3, loss = 1.07848496
Iteration 4, loss = 0.86917110
Iteration 5, loss = 0.74425139
Iteration 6, loss = 0.69839333
Iteration 7, loss = 0.69810158
Iteration 8, loss = 0.72058526
Iteration 9, loss = 0.73271713
Iteration 10, loss = 0.72699927
Iteration 11, loss = 0.69997667
Iteration 12, loss = 0.66889283
Iteration 13, loss = 0.63935658
Iteration 14, loss = 0.61980360
Iteration 15, loss = 0.60750587
Iteration 16, loss = 0.59951252
Iteration 17, loss = 0.59699883
Iteration 18, loss = 0.59543313
Iteration 19, loss = 0.59157277
Iteration 20, loss = 0.58645572
Iteration 21, loss = 0.57920591
Iteration 22, loss = 0.57197468
Iteration 23, loss = 0.56277398
Iteration 24, loss = 0.55555745
Iteration 25, loss = 0.54841008
Iteration 26, loss = 0.54418376
Iteration 27, loss = 0.53942495
Iteration 28, loss = 0.53565860
Iteration 29, loss = 0.53076002
Iteration 30, loss = 0.52511533
Iteration 31, loss = 0.51986969
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [35]:
print(f'R2 Score: {r2_score(y_test, pipe.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2 Score: -0.3082778923399605
MAE: 1.0490446037874404


In [36]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 1.32251320
Iteration 2, loss = 0.97597560
Iteration 3, loss = 0.76857794
Iteration 4, loss = 0.69117572
Iteration 5, loss = 0.72242077
Iteration 6, loss = 0.72723562
Iteration 7, loss = 0.68350198
Iteration 8, loss = 0.63291571
Iteration 9, loss = 0.61955937
Iteration 10, loss = 0.62171366
Iteration 11, loss = 0.62217096
Iteration 12, loss = 0.61045266
Iteration 13, loss = 0.59236066
Iteration 14, loss = 0.57843077
Iteration 15, loss = 0.56470634
Iteration 16, loss = 0.56098445
Iteration 17, loss = 0.55418032
Iteration 18, loss = 0.54341892
Iteration 19, loss = 0.53168244
Iteration 20, loss = 0.52409951
Iteration 21, loss = 0.51514029
Iteration 22, loss = 0.50227225
Iteration 23, loss = 0.48917023
Iteration 24, loss = 0.47803121
Iteration 25, loss = 0.46808886
Iteration 26, loss = 0.45295547
Iteration 27, loss = 0.44276363
Iteration 28, loss = 0.42691422
Iteration 29, loss = 0.41228769
Iteration 30, loss = 0.40013697
Iteration 31, loss = 0.38518230
Iteration 32, los

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [37]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.40415061614712977
MAE: 1.0578954944850267


In [38]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (60, 60, 60),
                            alpha = 24))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 18.41970730
Iteration 2, loss = 17.81082177
Iteration 3, loss = 17.25519245
Iteration 4, loss = 16.73978470
Iteration 5, loss = 16.27144597
Iteration 6, loss = 15.82947054
Iteration 7, loss = 15.40842903
Iteration 8, loss = 14.99669550
Iteration 9, loss = 14.58894312
Iteration 10, loss = 14.18705597
Iteration 11, loss = 13.79240859
Iteration 12, loss = 13.40794952
Iteration 13, loss = 13.03240163
Iteration 14, loss = 12.66869991
Iteration 15, loss = 12.31611718
Iteration 16, loss = 11.97386540
Iteration 17, loss = 11.64026187
Iteration 18, loss = 11.31541974
Iteration 19, loss = 10.99877410
Iteration 20, loss = 10.69038244
Iteration 21, loss = 10.39035306
Iteration 22, loss = 10.09794551
Iteration 23, loss = 9.81422545
Iteration 24, loss = 9.53840636
Iteration 25, loss = 9.27063075
Iteration 26, loss = 9.01030174
Iteration 27, loss = 8.75781966
Iteration 28, loss = 8.51199114
Iteration 29, loss = 8.27331572
Iteration 30, loss = 8.04136521
Iteration 31, loss = 7.8163

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=24, hidden_layer_sizes=(60, 60, 60),
                              verbose=True))])

In [39]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.01980533449685784
MAE: 0.94583971639177


In [40]:
pd.set_option('display.max_columns', None)
data2 = pd.read_csv('../data/data_teams.csv')
data2.head(1)

,Date,Time,TeamH,TeamA,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HStr,HSOH,HSOA,HSAH,HSAA,HSDH,HSDA,HO,HA,HM,HD,HDP,HIP,HSAAg,HATAAg,HGls,HAssts,HGlsPK,HxG,HNpxG,HxAG,HPoss,HPrg,HPrgP,HGKS%,HGKCS%,HSoT,Hg-xG,HCmp,HCmp%,HPrgDist,HXa,HKP,HPPA,HCrsPA,HTB,HCrs,HCK,HSCA,HGCA,HTkIW,HLost,HBlocks,HPass,HInt,HClr,HAttPen,HSucc%,HCPA,HDis,HPrgR,HOnG,HOnGA,HOnxG,HOnxGA,AStr,ASOH,ASOA,ASAH,ASAA,ASDH,ASDA,AO,AA,AM,AD,ADP,AIP,ASAAg,ATAAg,AGls,AAssts,AGlsPK,AxG,ANpxG,AxAG,APoss,APrg,APrgP,AGKS%,AGKCS%,ASoT,Ag-xG,ACmp,ACmp%,APrgDist,AXa,AKP,APPA,ACrsPA,ATB,ACrs,ACK,ASCA,AGCA,ATkIW,ALost,ABlocks,APass,AInt,AClr,AAttPen,ASucc%,ACPA,ADis,APrgR,AOnG,AOnGA,AOnxG,AOnxGA
0,5/8/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,3,1085,1100,1110,1110,1060,1090,76,77,75,76,4,2,26.6,25.2,1.0,0.76,0.97,1.02,0.96,0.75,45.8,14.2,32.2,71.8,24.3,3.51,-0.02,348.8,77.2,2310.4,0.61,8.19,6.14,1.27,0.95,15.4,4.89,19.4,1.76,11.2,6.73,12.3,9.08,9.65,21.5,19.9,46.9,5.41,12.1,32.1,1.05,1.3,1.02,1.29,4,1245,1285,1250,1250,1240,1320,80,83,81,79,7,7,24.1,23.0,2.14,1.62,2.05,1.87,1.79,1.39,59.5,21.7,54.2,70.6,35.1,5.03,0.27,482.8,83.1,2658.9,1.21,11.7,12.01,1.65,2.24,17.8,5.81,27.6,3.81,8.89,6.51,9.49,7.22,6.35,15.8,34.0,46.3,7.38,10.1,53.5,2.24,1.16,1.87,1.12


In [41]:
X = data2[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HDP', 'HIP', 'HSAAg', 'HATAAg', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG',
          'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA',
          'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA',
          'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA',
          'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP',
          'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK',
          'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr','AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG',
          'AOnGA', 'AOnxG', 'AOnxGA']].copy()
y = data2['FTHG']

In [42]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [44]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 2.38202183
Iteration 2, loss = 1.60530599
Iteration 3, loss = 1.14837889
Iteration 4, loss = 1.01022658
Iteration 5, loss = 1.03695999
Iteration 6, loss = 1.08257795
Iteration 7, loss = 1.05578065
Iteration 8, loss = 0.97883276
Iteration 9, loss = 0.89449845
Iteration 10, loss = 0.86003736
Iteration 11, loss = 0.83279136
Iteration 12, loss = 0.82385383
Iteration 13, loss = 0.81303900
Iteration 14, loss = 0.79737920
Iteration 15, loss = 0.78074456
Iteration 16, loss = 0.76857685
Iteration 17, loss = 0.76102930
Iteration 18, loss = 0.75552084
Iteration 19, loss = 0.75513947
Iteration 20, loss = 0.74845139
Iteration 21, loss = 0.73932726
Iteration 22, loss = 0.73222409
Iteration 23, loss = 0.72419612
Iteration 24, loss = 0.71750917
Iteration 25, loss = 0.71076769
Iteration 26, loss = 0.70421155
Iteration 27, loss = 0.69789681
Iteration 28, loss = 0.69175890
Iteration 29, loss = 0.68713249
Iteration 30, loss = 0.68239994
Iteration 31, loss = 0.67779328
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [45]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.18706802552386836
MAE: 1.2400045019608155


In [46]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 1.80419888
Iteration 2, loss = 1.22806265
Iteration 3, loss = 0.94231595
Iteration 4, loss = 1.02764890
Iteration 5, loss = 1.00974655
Iteration 6, loss = 0.86945267
Iteration 7, loss = 0.82713210
Iteration 8, loss = 0.83647798
Iteration 9, loss = 0.82791580
Iteration 10, loss = 0.79001268
Iteration 11, loss = 0.74942402
Iteration 12, loss = 0.75235380
Iteration 13, loss = 0.74662886
Iteration 14, loss = 0.72314251
Iteration 15, loss = 0.71151105
Iteration 16, loss = 0.70532886
Iteration 17, loss = 0.69342073
Iteration 18, loss = 0.67926009
Iteration 19, loss = 0.66263776
Iteration 20, loss = 0.64977215
Iteration 21, loss = 0.63706475
Iteration 22, loss = 0.62543024
Iteration 23, loss = 0.61367489
Iteration 24, loss = 0.59954091
Iteration 25, loss = 0.58417164
Iteration 26, loss = 0.57689159
Iteration 27, loss = 0.56292361
Iteration 28, loss = 0.54943971
Iteration 29, loss = 0.53790055
Iteration 30, loss = 0.52227109
Iteration 31, loss = 0.50937432
Iteration 32, los

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [47]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.33326936627280257
MAE: 1.3355596461935433


In [48]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (60, 60, 60),
                            alpha = 24))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 19.68686555
Iteration 2, loss = 18.86251254
Iteration 3, loss = 18.16314915
Iteration 4, loss = 17.58202667
Iteration 5, loss = 17.09807334
Iteration 6, loss = 16.66642078
Iteration 7, loss = 16.24972995
Iteration 8, loss = 15.82292132
Iteration 9, loss = 15.39046115
Iteration 10, loss = 14.96615954
Iteration 11, loss = 14.56301427
Iteration 12, loss = 14.17457712
Iteration 13, loss = 13.79729807
Iteration 14, loss = 13.42839137
Iteration 15, loss = 13.06485604
Iteration 16, loss = 12.70864220
Iteration 17, loss = 12.36224374
Iteration 18, loss = 12.02640844
Iteration 19, loss = 11.70177158
Iteration 20, loss = 11.38557687
Iteration 21, loss = 11.07790274
Iteration 22, loss = 10.77854602
Iteration 23, loss = 10.48555710
Iteration 24, loss = 10.20047970
Iteration 25, loss = 9.92482696
Iteration 26, loss = 9.65527856
Iteration 27, loss = 9.39446149
Iteration 28, loss = 9.14109614
Iteration 29, loss = 8.89379290
Iteration 30, loss = 8.65430454
Iteration 31, loss = 8.42

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=24, hidden_layer_sizes=(60, 60, 60),
                              verbose=True))])

In [49]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: 0.14431889950283006
MAE: 1.0661958144663661


In [50]:
X = data2[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HDP', 'HIP', 'HSAAg', 'HATAAg', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG',
          'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA',
          'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA',
          'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA',
          'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP',
          'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK',
          'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr','AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG',
          'AOnGA', 'AOnxG', 'AOnxGA']].copy()
y = data2['FTAG']

In [51]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [53]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 1.98332060
Iteration 2, loss = 1.16816248
Iteration 3, loss = 0.80487271
Iteration 4, loss = 0.74139499
Iteration 5, loss = 0.81407913
Iteration 6, loss = 0.83505979
Iteration 7, loss = 0.78262309
Iteration 8, loss = 0.70241701
Iteration 9, loss = 0.63896268
Iteration 10, loss = 0.61190881
Iteration 11, loss = 0.61658550
Iteration 12, loss = 0.61860470
Iteration 13, loss = 0.61490517
Iteration 14, loss = 0.60120693
Iteration 15, loss = 0.58189299
Iteration 16, loss = 0.56787884
Iteration 17, loss = 0.55682131
Iteration 18, loss = 0.55474947
Iteration 19, loss = 0.55105991
Iteration 20, loss = 0.54538303
Iteration 21, loss = 0.53565251
Iteration 22, loss = 0.52640468
Iteration 23, loss = 0.52079748
Iteration 24, loss = 0.51756700
Iteration 25, loss = 0.51556092
Iteration 26, loss = 0.50944101
Iteration 27, loss = 0.50288208
Iteration 28, loss = 0.49462139
Iteration 29, loss = 0.49047682
Iteration 30, loss = 0.48784017
Iteration 31, loss = 0.48343287
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [55]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.29189419599005784
MAE: 1.0504500646951493


In [56]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 1.05335695
Iteration 2, loss = 0.71849298
Iteration 3, loss = 0.71628524
Iteration 4, loss = 0.65616013
Iteration 5, loss = 0.60378485
Iteration 6, loss = 0.59266795
Iteration 7, loss = 0.58637894
Iteration 8, loss = 0.56481550
Iteration 9, loss = 0.54831031
Iteration 10, loss = 0.55381940
Iteration 11, loss = 0.54584130
Iteration 12, loss = 0.52865707
Iteration 13, loss = 0.52373964
Iteration 14, loss = 0.51335327
Iteration 15, loss = 0.50138429
Iteration 16, loss = 0.49486064
Iteration 17, loss = 0.48460291
Iteration 18, loss = 0.47428704
Iteration 19, loss = 0.46753603
Iteration 20, loss = 0.45641579
Iteration 21, loss = 0.44601506
Iteration 22, loss = 0.43574716
Iteration 23, loss = 0.42452852
Iteration 24, loss = 0.41388198
Iteration 25, loss = 0.40607182
Iteration 26, loss = 0.40468834
Iteration 27, loss = 0.39033603
Iteration 28, loss = 0.38150222
Iteration 29, loss = 0.36630167
Iteration 30, loss = 0.35648421
Iteration 31, loss = 0.34926782
Iteration 32, los

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [57]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.4009661147409531
MAE: 1.0930663757990982


In [58]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (60, 60, 60),
                            alpha = 24))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 19.22060745
Iteration 2, loss = 18.31203429
Iteration 3, loss = 17.64791502
Iteration 4, loss = 17.10886176
Iteration 5, loss = 16.66330313
Iteration 6, loss = 16.24817471
Iteration 7, loss = 15.84125747
Iteration 8, loss = 15.42471378
Iteration 9, loss = 15.00565841
Iteration 10, loss = 14.59537858
Iteration 11, loss = 14.19294263
Iteration 12, loss = 13.80865859
Iteration 13, loss = 13.43823824
Iteration 14, loss = 13.07651626
Iteration 15, loss = 12.72234631
Iteration 16, loss = 12.37543812
Iteration 17, loss = 12.03598569
Iteration 18, loss = 11.70355736
Iteration 19, loss = 11.38021477
Iteration 20, loss = 11.06726962
Iteration 21, loss = 10.76238793
Iteration 22, loss = 10.46539363
Iteration 23, loss = 10.17733127
Iteration 24, loss = 9.89722974
Iteration 25, loss = 9.62447839
Iteration 26, loss = 9.35968238
Iteration 27, loss = 9.10161823
Iteration 28, loss = 8.85079424
Iteration 29, loss = 8.60706031
Iteration 30, loss = 8.37005984
Iteration 31, loss = 8.139

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=24, hidden_layer_sizes=(60, 60, 60),
                              verbose=True))])

In [59]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: 0.015915146267695968
MAE: 0.9319089886077196


In [60]:
pd.set_option('display.max_columns', None)
data3 = pd.read_csv('../data/final_data_corrected.csv')
data3.head(1)

,Date,Time,TeamH,TeamA,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,HStr,HSOH,HSOA,HSAH,HSAA,HSDH,HSDA,HO,HA,HM,HD,HDP,HIP,HSAAg,HATAAg,HGls,HAssts,HGlsPK,HxG,HNpxG,HxAG,HPoss,HPrg,HPrgP,HGKS%,HGKCS%,HSoT,Hg-xG,HCmp,HCmp%,HPrgDist,HXa,HKP,HPPA,HCrsPA,HTB,HCrs,HCK,HSCA,HGCA,HTkIW,HLost,HBlocks,HPass,HInt,HClr,HAttPen,HSucc%,HCPA,HDis,HPrgR,HOnG,HOnGA,HOnxG,HOnxGA,AStr,ASOH,ASOA,ASAH,ASAA,ASDH,ASDA,AO,AA,AM,AD,ADP,AIP,ASAAg,ATAAg,AGls,AAssts,AGlsPK,AxG,ANpxG,AxAG,APoss,APrg,APrgP,AGKS%,AGKCS%,ASoT,Ag-xG,ACmp,ACmp%,APrgDist,AXa,AKP,APPA,ACrsPA,ATB,ACrs,ACK,ASCA,AGCA,ATkIW,ALost,ABlocks,APass,AInt,AClr,AAttPen,ASucc%,ACPA,ADis,APrgR,AOnG,AOnGA,AOnxG,AOnxGA
0,5/8/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,4.2,3.6,1.85,4.33,3.5,1.87,4.3,3.55,1.85,4.5,3.65,1.89,4.4,3.5,1.83,4.6,3.5,1.87,4.6,3.78,1.95,4.39,3.59,1.88,2.1,1.72,2.14,1.78,2.19,1.91,2.09,1.76,0.5,2.04,1.89,2.03,1.89,2.06,1.91,2.01,1.87,4.5,3.6,1.8,4.5,3.5,1.83,4.4,3.55,1.85,4.58,3.63,1.88,4.8,3.4,1.78,4.75,3.5,1.85,5.01,3.7,1.91,4.56,3.57,1.85,2.1,1.72,2.14,1.78,2.19,1.91,2.08,1.76,0.5,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85,3,1085,1100,1110,1110,1060,1090,76,77,75,76,4,2,26.6,25.2,1.0,0.76,0.97,1.02,0.96,0.75,45.8,14.2,32.2,71.8,24.3,3.51,-0.02,348.8,77.2,2310.4,0.61,8.19,6.14,1.27,0.95,15.4,4.89,19.4,1.76,11.2,6.73,12.3,9.08,9.65,21.5,19.9,46.9,5.41,12.1,32.1,1.05,1.3,1.02,1.29,4,1245,1285,1250,1250,1240,1320,80,83,81,79,7,7,24.1,23.0,2.14,1.62,2.05,1.87,1.79,1.39,59.5,21.7,54.2,70.6,35.1,5.03,0.27,482.8,83.1,2658.9,1.21,11.7,12.01,1.65,2.24,17.8,5.81,27.6,3.81,8.89,6.51,9.49,7.22,6.35,15.8,34.0,46.3,7.38,10.1,53.5,2.24,1.16,1.87,1.12


In [61]:
data_basic = data3.drop(['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
          'PSH', 'PSD', 'PSA', 'WHH','WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH', 'AvgD', 'AvgA', 'B365>2.5',
          'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH','B365AHA', 'PAHH', 'PAHA', 
          'MaxAHH', 'MaxAHA', "AvgAHH", 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA',
          'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD',
          'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH',
          'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG',
          'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA',
          'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA',
          'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA',
          'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP',
          'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK',
          'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr','AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG',
          'AOnGA', 'AOnxG', 'AOnxGA'], axis=1)
data_basic.head(1)

,Date,Time,TeamH,TeamA,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HStr,HSOH,HSOA,HSAH,HSAA,HSDH,HSDA,HO,HA,HM,HD,HDP,HIP,HSAAg,HATAAg
0,5/8/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,3,1085,1100,1110,1110,1060,1090,76,77,75,76,4,2,26.6,25.2


In [62]:
X = data3[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HSAAg', 'HATAAg']].copy()
y = data3['FTHG']

In [63]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [65]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 2.71856330
Iteration 2, loss = 2.50876091
Iteration 3, loss = 2.32493582
Iteration 4, loss = 2.14079677
Iteration 5, loss = 1.97900873
Iteration 6, loss = 1.82335038
Iteration 7, loss = 1.68228978
Iteration 8, loss = 1.55114550
Iteration 9, loss = 1.42687810
Iteration 10, loss = 1.31729881
Iteration 11, loss = 1.22092737
Iteration 12, loss = 1.12803022
Iteration 13, loss = 1.05729919
Iteration 14, loss = 0.99126398
Iteration 15, loss = 0.94144926
Iteration 16, loss = 0.90167279
Iteration 17, loss = 0.86889301
Iteration 18, loss = 0.85647914
Iteration 19, loss = 0.84354015
Iteration 20, loss = 0.83604724
Iteration 21, loss = 0.83003238
Iteration 22, loss = 0.82645333
Iteration 23, loss = 0.82053111
Iteration 24, loss = 0.81352325
Iteration 25, loss = 0.80358003
Iteration 26, loss = 0.79177162
Iteration 27, loss = 0.78050601
Iteration 28, loss = 0.76916114
Iteration 29, loss = 0.75910277
Iteration 30, loss = 0.74887673
Iteration 31, loss = 0.74175378
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [67]:
print(f'R2 Score: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2 Score: -0.18651078618578132
MAE: 1.1982265008836679


In [68]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 1.76107335
Iteration 2, loss = 1.39893535
Iteration 3, loss = 1.14255632
Iteration 4, loss = 0.96117267
Iteration 5, loss = 0.88394017
Iteration 6, loss = 0.91797949
Iteration 7, loss = 0.93232030
Iteration 8, loss = 0.89665445
Iteration 9, loss = 0.84941000
Iteration 10, loss = 0.81921157
Iteration 11, loss = 0.80411101
Iteration 12, loss = 0.79487962
Iteration 13, loss = 0.78608790
Iteration 14, loss = 0.77150660
Iteration 15, loss = 0.75662436
Iteration 16, loss = 0.73692983
Iteration 17, loss = 0.72295212
Iteration 18, loss = 0.70757257
Iteration 19, loss = 0.69564164
Iteration 20, loss = 0.68120591
Iteration 21, loss = 0.66621080
Iteration 22, loss = 0.65240110
Iteration 23, loss = 0.63680957
Iteration 24, loss = 0.61978070
Iteration 25, loss = 0.60841652
Iteration 26, loss = 0.59086490
Iteration 27, loss = 0.57436193
Iteration 28, loss = 0.55574213
Iteration 29, loss = 0.54250812
Iteration 30, loss = 0.52414250
Iteration 31, loss = 0.50680227
Iteration 32, los

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [70]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.25318800342171177
MAE: 1.2369927825046516


In [71]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (60, 60, 60),
                            alpha = 24))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 18.46748757
Iteration 2, loss = 17.88584166
Iteration 3, loss = 17.33067068
Iteration 4, loss = 16.81109672
Iteration 5, loss = 16.30672623
Iteration 6, loss = 15.82341797
Iteration 7, loss = 15.35876859
Iteration 8, loss = 14.91354861
Iteration 9, loss = 14.50006977
Iteration 10, loss = 14.09681387
Iteration 11, loss = 13.71881131
Iteration 12, loss = 13.36521123
Iteration 13, loss = 13.03425265
Iteration 14, loss = 12.72032759
Iteration 15, loss = 12.41839537
Iteration 16, loss = 12.12531797
Iteration 17, loss = 11.83969300
Iteration 18, loss = 11.55667813
Iteration 19, loss = 11.27742698
Iteration 20, loss = 11.00275181
Iteration 21, loss = 10.73742500
Iteration 22, loss = 10.47402255
Iteration 23, loss = 10.22021195
Iteration 24, loss = 9.97358812
Iteration 25, loss = 9.73353105
Iteration 26, loss = 9.50029191
Iteration 27, loss = 9.27314331
Iteration 28, loss = 9.05171071
Iteration 29, loss = 8.83560740
Iteration 30, loss = 8.62479155
Iteration 31, loss = 8.419

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=24, hidden_layer_sizes=(60, 60, 60),
                              verbose=True))])

In [72]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: 0.07946736813931121
MAE: 1.132521243291272


In [73]:
X = data3[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HSAAg', 'HATAAg']].copy()
y = data3['FTAG']

In [74]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [76]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 1.43718209
Iteration 2, loss = 1.30927718
Iteration 3, loss = 1.20139949
Iteration 4, loss = 1.10239431
Iteration 5, loss = 1.02871818
Iteration 6, loss = 0.95833280
Iteration 7, loss = 0.90680687
Iteration 8, loss = 0.86601573
Iteration 9, loss = 0.83377091
Iteration 10, loss = 0.80866275
Iteration 11, loss = 0.79353030
Iteration 12, loss = 0.77940252
Iteration 13, loss = 0.76731922
Iteration 14, loss = 0.75569677
Iteration 15, loss = 0.74183285
Iteration 16, loss = 0.72796157
Iteration 17, loss = 0.71263072
Iteration 18, loss = 0.69804921
Iteration 19, loss = 0.68121869
Iteration 20, loss = 0.66736085
Iteration 21, loss = 0.65291478
Iteration 22, loss = 0.64018625
Iteration 23, loss = 0.62823321
Iteration 24, loss = 0.61741749
Iteration 25, loss = 0.60582027
Iteration 26, loss = 0.59574358
Iteration 27, loss = 0.58524111
Iteration 28, loss = 0.57538131
Iteration 29, loss = 0.56479391
Iteration 30, loss = 0.55607540
Iteration 31, loss = 0.54536707
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [77]:
print(f'R2 Score: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2 Score: -0.2970217085456963
MAE: 1.0504018620781537


In [78]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 2.07098654
Iteration 2, loss = 1.69418696
Iteration 3, loss = 1.40559963
Iteration 4, loss = 1.17115168
Iteration 5, loss = 0.98443038
Iteration 6, loss = 0.84958999
Iteration 7, loss = 0.77901318
Iteration 8, loss = 0.77463233
Iteration 9, loss = 0.80128263
Iteration 10, loss = 0.79937449
Iteration 11, loss = 0.76379513
Iteration 12, loss = 0.71610861
Iteration 13, loss = 0.68102960
Iteration 14, loss = 0.65940162
Iteration 15, loss = 0.64293076
Iteration 16, loss = 0.63141313
Iteration 17, loss = 0.61746834
Iteration 18, loss = 0.60069814
Iteration 19, loss = 0.58099441
Iteration 20, loss = 0.56652810
Iteration 21, loss = 0.54956818
Iteration 22, loss = 0.53752699
Iteration 23, loss = 0.52291627
Iteration 24, loss = 0.50753470
Iteration 25, loss = 0.49185661
Iteration 26, loss = 0.47701288
Iteration 27, loss = 0.46282216
Iteration 28, loss = 0.45007526
Iteration 29, loss = 0.43479541
Iteration 30, loss = 0.42134530
Iteration 31, loss = 0.40982527
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 58, loss = 0.12349805
Iteration 59, loss = 0.11459186
Iteration 60, loss = 0.10457845
Iteration 61, loss = 0.09775084
Iteration 62, loss = 0.08841852
Iteration 63, loss = 0.08065044
Iteration 64, loss = 0.07186124
Iteration 65, loss = 0.06550596
Iteration 66, loss = 0.05869866
Iteration 67, loss = 0.05214626
Iteration 68, loss = 0.04621384
Iteration 69, loss = 0.04097441
Iteration 70, loss = 0.03626797
Iteration 71, loss = 0.03197173
Iteration 72, loss = 0.02818504
Iteration 73, loss = 0.02481082
Iteration 74, loss = 0.02204204
Iteration 75, loss = 0.01938259
Iteration 76, loss = 0.01711875
Iteration 77, loss = 0.01497268
Iteration 78, loss = 0.01317245
Iteration 79, loss = 0.01165078
Iteration 80, loss = 0.01040567
Iteration 81, loss = 0.00931183
Iteration 82, loss = 0.00806401
Iteration 83, loss = 0.00749602
Iteration 84, loss = 0.00645933
Iteration 85, loss = 0.00578085
Iteration 86, loss = 0.00521239
Iteration 87, loss = 0.00462487
Iteration 88, loss = 0.00397385
Iteratio

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [79]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.41777532870319
MAE: 1.1059563245162438


In [80]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (60, 60, 60),
                            alpha = 24))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 16.96038259
Iteration 2, loss = 16.50033268
Iteration 3, loss = 16.06947688
Iteration 4, loss = 15.65443461
Iteration 5, loss = 15.25562856
Iteration 6, loss = 14.87273716
Iteration 7, loss = 14.50016348
Iteration 8, loss = 14.13930879
Iteration 9, loss = 13.78613531
Iteration 10, loss = 13.44209300
Iteration 11, loss = 13.10385458
Iteration 12, loss = 12.77321317
Iteration 13, loss = 12.44971087
Iteration 14, loss = 12.13359719
Iteration 15, loss = 11.82510754
Iteration 16, loss = 11.52441328
Iteration 17, loss = 11.23100011
Iteration 18, loss = 10.94526236
Iteration 19, loss = 10.66639674
Iteration 20, loss = 10.39473535
Iteration 21, loss = 10.12949399
Iteration 22, loss = 9.87085774
Iteration 23, loss = 9.61854333
Iteration 24, loss = 9.37269654
Iteration 25, loss = 9.13308875
Iteration 26, loss = 8.89944251
Iteration 27, loss = 8.67238438
Iteration 28, loss = 8.45093318
Iteration 29, loss = 8.23518675
Iteration 30, loss = 8.02447287
Iteration 31, loss = 7.81951

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=24, hidden_layer_sizes=(60, 60, 60),
                              verbose=True))])

In [81]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.02069332726621198
MAE: 0.9451776475618091


In [104]:
pd.set_option('display.max_columns', None)
data_winner = pd.read_csv('../data/final_data_corrected.csv')
data_winner.head(1)

,Date,Time,TeamH,TeamA,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,HStr,HSOH,HSOA,HSAH,HSAA,HSDH,HSDA,HO,HA,HM,HD,HDP,HIP,HSAAg,HATAAg,HGls,HAssts,HGlsPK,HxG,HNpxG,HxAG,HPoss,HPrg,HPrgP,HGKS%,HGKCS%,HSoT,Hg-xG,HCmp,HCmp%,HPrgDist,HXa,HKP,HPPA,HCrsPA,HTB,HCrs,HCK,HSCA,HGCA,HTkIW,HLost,HBlocks,HPass,HInt,HClr,HAttPen,HSucc%,HCPA,HDis,HPrgR,HOnG,HOnGA,HOnxG,HOnxGA,AStr,ASOH,ASOA,ASAH,ASAA,ASDH,ASDA,AO,AA,AM,AD,ADP,AIP,ASAAg,ATAAg,AGls,AAssts,AGlsPK,AxG,ANpxG,AxAG,APoss,APrg,APrgP,AGKS%,AGKCS%,ASoT,Ag-xG,ACmp,ACmp%,APrgDist,AXa,AKP,APPA,ACrsPA,ATB,ACrs,ACK,ASCA,AGCA,ATkIW,ALost,ABlocks,APass,AInt,AClr,AAttPen,ASucc%,ACPA,ADis,APrgR,AOnG,AOnGA,AOnxG,AOnxGA
0,5/8/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,4.2,3.6,1.85,4.33,3.5,1.87,4.3,3.55,1.85,4.5,3.65,1.89,4.4,3.5,1.83,4.6,3.5,1.87,4.6,3.78,1.95,4.39,3.59,1.88,2.1,1.72,2.14,1.78,2.19,1.91,2.09,1.76,0.5,2.04,1.89,2.03,1.89,2.06,1.91,2.01,1.87,4.5,3.6,1.8,4.5,3.5,1.83,4.4,3.55,1.85,4.58,3.63,1.88,4.8,3.4,1.78,4.75,3.5,1.85,5.01,3.7,1.91,4.56,3.57,1.85,2.1,1.72,2.14,1.78,2.19,1.91,2.08,1.76,0.5,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85,3,1085,1100,1110,1110,1060,1090,76,77,75,76,4,2,26.6,25.2,1.0,0.76,0.97,1.02,0.96,0.75,45.8,14.2,32.2,71.8,24.3,3.51,-0.02,348.8,77.2,2310.4,0.61,8.19,6.14,1.27,0.95,15.4,4.89,19.4,1.76,11.2,6.73,12.3,9.08,9.65,21.5,19.9,46.9,5.41,12.1,32.1,1.05,1.3,1.02,1.29,4,1245,1285,1250,1250,1240,1320,80,83,81,79,7,7,24.1,23.0,2.14,1.62,2.05,1.87,1.79,1.39,59.5,21.7,54.2,70.6,35.1,5.03,0.27,482.8,83.1,2658.9,1.21,11.7,12.01,1.65,2.24,17.8,5.81,27.6,3.81,8.89,6.51,9.49,7.22,6.35,15.8,34.0,46.3,7.38,10.1,53.5,2.24,1.16,1.87,1.12


In [105]:
data_winner.head(1)

,Date,Time,TeamH,TeamA,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,HStr,HSOH,HSOA,HSAH,HSAA,HSDH,HSDA,HO,HA,HM,HD,HDP,HIP,HSAAg,HATAAg,HGls,HAssts,HGlsPK,HxG,HNpxG,HxAG,HPoss,HPrg,HPrgP,HGKS%,HGKCS%,HSoT,Hg-xG,HCmp,HCmp%,HPrgDist,HXa,HKP,HPPA,HCrsPA,HTB,HCrs,HCK,HSCA,HGCA,HTkIW,HLost,HBlocks,HPass,HInt,HClr,HAttPen,HSucc%,HCPA,HDis,HPrgR,HOnG,HOnGA,HOnxG,HOnxGA,AStr,ASOH,ASOA,ASAH,ASAA,ASDH,ASDA,AO,AA,AM,AD,ADP,AIP,ASAAg,ATAAg,AGls,AAssts,AGlsPK,AxG,ANpxG,AxAG,APoss,APrg,APrgP,AGKS%,AGKCS%,ASoT,Ag-xG,ACmp,ACmp%,APrgDist,AXa,AKP,APPA,ACrsPA,ATB,ACrs,ACK,ASCA,AGCA,ATkIW,ALost,ABlocks,APass,AInt,AClr,AAttPen,ASucc%,ACPA,ADis,APrgR,AOnG,AOnGA,AOnxG,AOnxGA
0,5/8/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,4.2,3.6,1.85,4.33,3.5,1.87,4.3,3.55,1.85,4.5,3.65,1.89,4.4,3.5,1.83,4.6,3.5,1.87,4.6,3.78,1.95,4.39,3.59,1.88,2.1,1.72,2.14,1.78,2.19,1.91,2.09,1.76,0.5,2.04,1.89,2.03,1.89,2.06,1.91,2.01,1.87,4.5,3.6,1.8,4.5,3.5,1.83,4.4,3.55,1.85,4.58,3.63,1.88,4.8,3.4,1.78,4.75,3.5,1.85,5.01,3.7,1.91,4.56,3.57,1.85,2.1,1.72,2.14,1.78,2.19,1.91,2.08,1.76,0.5,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85,3,1085,1100,1110,1110,1060,1090,76,77,75,76,4,2,26.6,25.2,1.0,0.76,0.97,1.02,0.96,0.75,45.8,14.2,32.2,71.8,24.3,3.51,-0.02,348.8,77.2,2310.4,0.61,8.19,6.14,1.27,0.95,15.4,4.89,19.4,1.76,11.2,6.73,12.3,9.08,9.65,21.5,19.9,46.9,5.41,12.1,32.1,1.05,1.3,1.02,1.29,4,1245,1285,1250,1250,1240,1320,80,83,81,79,7,7,24.1,23.0,2.14,1.62,2.05,1.87,1.79,1.39,59.5,21.7,54.2,70.6,35.1,5.03,0.27,482.8,83.1,2658.9,1.21,11.7,12.01,1.65,2.24,17.8,5.81,27.6,3.81,8.89,6.51,9.49,7.22,6.35,15.8,34.0,46.3,7.38,10.1,53.5,2.24,1.16,1.87,1.12


In [106]:
data_winner = data_winner.drop(['FTHG', 'FTAG', 'HTHG', 'HTAG', 'HTR'], axis=1)
data_winner.head(1)

,Date,Time,TeamH,TeamA,FTR,Referee,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,HStr,HSOH,HSOA,HSAH,HSAA,HSDH,HSDA,HO,HA,HM,HD,HDP,HIP,HSAAg,HATAAg,HGls,HAssts,HGlsPK,HxG,HNpxG,HxAG,HPoss,HPrg,HPrgP,HGKS%,HGKCS%,HSoT,Hg-xG,HCmp,HCmp%,HPrgDist,HXa,HKP,HPPA,HCrsPA,HTB,HCrs,HCK,HSCA,HGCA,HTkIW,HLost,HBlocks,HPass,HInt,HClr,HAttPen,HSucc%,HCPA,HDis,HPrgR,HOnG,HOnGA,HOnxG,HOnxGA,AStr,ASOH,ASOA,ASAH,ASAA,ASDH,ASDA,AO,AA,AM,AD,ADP,AIP,ASAAg,ATAAg,AGls,AAssts,AGlsPK,AxG,ANpxG,AxAG,APoss,APrg,APrgP,AGKS%,AGKCS%,ASoT,Ag-xG,ACmp,ACmp%,APrgDist,AXa,AKP,APPA,ACrsPA,ATB,ACrs,ACK,ASCA,AGCA,ATkIW,ALost,ABlocks,APass,AInt,AClr,AAttPen,ASucc%,ACPA,ADis,APrgR,AOnG,AOnGA,AOnxG,AOnxGA
0,5/8/2022,20:00,Crystal Palace,Arsenal,A,A Taylor,4.2,3.6,1.85,4.33,3.5,1.87,4.3,3.55,1.85,4.5,3.65,1.89,4.4,3.5,1.83,4.6,3.5,1.87,4.6,3.78,1.95,4.39,3.59,1.88,2.1,1.72,2.14,1.78,2.19,1.91,2.09,1.76,0.5,2.04,1.89,2.03,1.89,2.06,1.91,2.01,1.87,4.5,3.6,1.8,4.5,3.5,1.83,4.4,3.55,1.85,4.58,3.63,1.88,4.8,3.4,1.78,4.75,3.5,1.85,5.01,3.7,1.91,4.56,3.57,1.85,2.1,1.72,2.14,1.78,2.19,1.91,2.08,1.76,0.5,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85,3,1085,1100,1110,1110,1060,1090,76,77,75,76,4,2,26.6,25.2,1.0,0.76,0.97,1.02,0.96,0.75,45.8,14.2,32.2,71.8,24.3,3.51,-0.02,348.8,77.2,2310.4,0.61,8.19,6.14,1.27,0.95,15.4,4.89,19.4,1.76,11.2,6.73,12.3,9.08,9.65,21.5,19.9,46.9,5.41,12.1,32.1,1.05,1.3,1.02,1.29,4,1245,1285,1250,1250,1240,1320,80,83,81,79,7,7,24.1,23.0,2.14,1.62,2.05,1.87,1.79,1.39,59.5,21.7,54.2,70.6,35.1,5.03,0.27,482.8,83.1,2658.9,1.21,11.7,12.01,1.65,2.24,17.8,5.81,27.6,3.81,8.89,6.51,9.49,7.22,6.35,15.8,34.0,46.3,7.38,10.1,53.5,2.24,1.16,1.87,1.12


In [85]:
#data_winner['Date'] = pd.to_datetime(data_winner['Date'], dayfirst=True)
#data_winner['Time'] = pd.to_datetime(data_winner['Time'])

In [110]:
features = ['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
          'PSH', 'PSD', 'PSA', 'WHH','WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH', 'AvgD', 'AvgA', 'B365>2.5',
          'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH','B365AHA', 'PAHH', 'PAHA', 
          'MaxAHH', 'MaxAHA', "AvgAHH", 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA',
          'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD',
          'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH',
          'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HDP', 'HIP', 'HSAAg', 'HATAAg', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG',
          'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA',
          'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA',
          'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA',
          'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP',
          'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK',
          'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr','AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG',
          'AOnGA', 'AOnxG', 'AOnxGA']

X = StandardScaler().fit_transform(data_winner[features])
#X = OneHotEncoder(sparse_ouput = False, columns = ['TeamH', 'TeamA', 'Referee']) 
            
y = data_winner['FTR']

le = LabelEncoder().fit(y)
y = le.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 321) 

In [89]:
#Start with a sequential model
model = tf.keras.Sequential()

# Then add dense layer with relu activation
model.add(tf.keras.layers.Dense(units = 32,
                                input_shape = (len(features),),
                                activation = 'relu'))
# And end with an output layer with softmax activation (since this is a multiclass problem)
model.add(tf.keras.layers.Dense(3, activation = 'softmax'))

# Finally, compile the model
model.compile('adam', 'sparse_categorical_crossentropy')

In [90]:
model.fit(X_train, y_train, epochs = 100)

Epoch 1/100
9/9 [==============================] - 0s 3ms/step - loss: 1.1348
Epoch 2/100
9/9 [==============================] - 0s 733us/step - loss: 0.9884
Epoch 3/100
9/9 [==============================] - 0s 948us/step - loss: 0.9374
Epoch 4/100
9/9 [==============================] - 0s 1ms/step - loss: 0.9057
Epoch 5/100
9/9 [==============================] - 0s 938us/step - loss: 0.8785
Epoch 6/100
9/9 [==============================] - 0s 941us/step - loss: 0.8433
Epoch 7/100
9/9 [==============================] - 0s 635us/step - loss: 0.8254
Epoch 8/100
9/9 [==============================] - 0s 2ms/step - loss: 0.8038
Epoch 9/100
9/9 [==============================] - 0s 2ms/step - loss: 0.7883
Epoch 10/100
9/9 [==============================] - 0s 367us/step - loss: 0.7658
Epoch 11/100
9/9 [==============================] - 0s 996us/step - loss: 0.7552
Epoch 12/100
9/9 [==============================] - 0s 2ms/step - loss: 0.7419
Epoch 13/100
9/9 [=============================

In [91]:
model.predict(X_test)

3/3 [==============================] - 0s 2ms/step


array([[1.07096642e-01, 9.98476371e-02, 7.93055773e-01],
       [9.10652161e-01, 8.84968415e-02, 8.50929180e-04],
       [2.61472225e-01, 1.49766013e-01, 5.88761747e-01],
       [2.24758297e-01, 1.62662148e-01, 6.12579644e-01],
       [8.63515772e-04, 6.98541880e-01, 3.00594598e-01],
       [8.58711004e-01, 5.23547754e-02, 8.89342204e-02],
       [2.17412055e-01, 6.25987291e-01, 1.56600699e-01],
       [9.62834597e-01, 2.79401727e-02, 9.22528561e-03],
       [9.12244737e-01, 3.80007923e-02, 4.97543402e-02],
       [1.66124552e-02, 3.14251631e-02, 9.51962411e-01],
       [1.16907023e-01, 1.10082095e-02, 8.72084737e-01],
       [1.99468866e-01, 5.42932153e-01, 2.57598937e-01],
       [9.87810194e-02, 4.08028588e-02, 8.60416174e-01],
       [1.09669447e-01, 1.54648170e-01, 7.35682368e-01],
       [5.71290590e-03, 2.35282606e-03, 9.91934299e-01],
       [1.04031302e-02, 5.49347773e-02, 9.34662104e-01],
       [1.15128852e-01, 5.41212559e-02, 8.30749869e-01],
       [2.85403848e-01, 6.60694

In [92]:
y_pred = np.argmax(model.predict(X_test), axis = 1)
y_pred

3/3 [==============================] - 0s 1ms/step


array([2, 0, 2, 2, 1, 0, 1, 0, 0, 2, 2, 1, 2, 2, 2, 2, 2, 1, 0, 2, 1, 2,
       0, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 0, 0, 2, 2, 2, 0, 0, 2, 2,
       2, 2, 2, 0, 0, 2, 2, 2, 0, 2, 1, 1, 1, 0, 0, 2, 2, 0, 2, 0, 2, 2,
       2, 1, 1, 1, 2, 2, 0, 1, 1, 2, 0, 1, 2, 2, 2, 1, 2, 2, 2, 0, 0, 2,
       0, 2, 0, 0, 1, 0, 1], dtype=int64)

In [93]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.46      0.44      0.45        27
           1       0.37      0.32      0.34        22
           2       0.64      0.70      0.67        46

    accuracy                           0.54        95
   macro avg       0.49      0.49      0.49        95
weighted avg       0.53      0.54      0.53        95



In [94]:
confusion_matrix(y_test, y_pred)

array([[12,  7,  8],
       [ 5,  7, 10],
       [ 9,  5, 32]], dtype=int64)

In [95]:
pd.set_option('display.max_columns', None)
data_winner1 = pd.read_csv('../data/data_betting.csv')
data_winner1.head(1)

,Unnamed: 0,Date,Time,TeamH,TeamA,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,0,5/8/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,0,0,4.2,3.6,1.85,4.33,3.5,1.87,4.3,3.55,1.85,4.5,3.65,1.89,4.4,3.5,1.83,4.6,3.5,1.87,4.6,3.78,1.95,4.39,3.59,1.88,2.1,1.72,2.14,1.78,2.19,1.91,2.09,1.76,0.5,2.04,1.89,2.03,1.89,2.06,1.91,2.01,1.87,4.5,3.6,1.8,4.5,3.5,1.83,4.4,3.55,1.85,4.58,3.63,1.88,4.8,3.4,1.78,4.75,3.5,1.85,5.01,3.7,1.91,4.56,3.57,1.85,2.1,1.72,2.14,1.78,2.19,1.91,2.08,1.76,0.5,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85


In [96]:
data_winner1 = data_winner1.drop(['Unnamed: 0', 'FTHG', 'FTAG', 'HTHG', 'HTAG', 'HTR'], axis=1)
data_winner1.head()

,Date,Time,TeamH,TeamA,FTR,Referee,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,5/8/2022,20:00,Crystal Palace,Arsenal,A,A Taylor,0,0,4.20,3.6,1.85,4.33,3.50,1.87,4.30,3.55,1.85,4.50,3.65,1.89,4.40,3.5,1.83,4.60,3.5,1.87,4.60,3.78,1.95,4.39,3.59,1.88,2.10,1.72,2.14,1.78,2.19,1.91,2.09,1.76,0.50,2.04,1.89,2.03,1.89,2.06,1.91,2.01,1.87,4.50,3.60,1.80,4.50,3.5,1.83,4.40,3.55,1.85,4.58,3.63,1.88,4.80,3.4,1.78,4.75,3.5,1.85,5.01,3.70,1.91,4.56,3.57,1.85,2.10,1.72,2.14,1.78,2.19,1.91,2.08,1.76,0.50,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85
1,6/8/2022,12:30,Fulham,Liverpool,D,A Madley,0,0,11.00,6.0,1.25,10.00,5.75,1.28,12.00,5.75,1.27,11.20,6.22,1.28,12.00,5.5,1.27,13.00,6.0,1.25,13.00,6.40,1.31,10.99,6.05,1.28,1.50,2.62,1.50,2.70,1.54,2.76,1.48,2.63,1.75,1.90,2.03,1.91,2.00,1.92,2.04,1.89,1.99,11.00,5.75,1.28,9.25,6.0,1.29,11.00,5.50,1.30,10.50,6.50,1.29,11.00,5.5,1.27,11.50,6.0,1.29,11.95,6.93,1.30,10.33,6.20,1.28,1.50,2.62,1.49,2.77,1.51,3.00,1.47,2.73,1.75,1.90,2.03,1.91,2.02,2.01,2.06,1.89,1.99
2,6/8/2022,15:00,Bournemouth,Aston Villa,H,P Bankes,0,0,3.75,3.5,2.00,3.75,3.40,2.00,3.65,3.45,2.05,3.93,3.58,2.04,3.75,3.3,2.05,3.75,3.3,2.00,4.00,3.66,2.10,3.80,3.50,2.04,2.00,1.80,2.10,1.81,2.10,1.87,2.03,1.80,0.50,1.87,2.06,1.88,2.04,1.88,2.07,1.85,2.04,4.00,3.50,1.95,3.90,3.4,1.95,3.85,3.45,2.00,4.09,3.59,2.00,4.00,3.4,1.95,4.10,3.4,2.00,4.25,3.63,2.06,3.99,3.49,2.00,2.10,1.72,2.13,1.79,2.24,1.81,2.10,1.76,0.50,1.93,2.00,1.93,2.00,1.94,2.04,1.88,2.00
3,6/8/2022,15:00,Leeds,Wolves,H,R Jones,0,0,2.25,3.4,3.20,2.30,3.30,2.95,2.30,3.30,3.15,2.39,3.33,3.30,2.25,3.3,3.20,2.30,3.2,3.10,2.42,3.54,3.30,2.34,3.34,3.18,2.05,1.85,2.09,1.83,2.11,1.87,2.03,1.81,-0.25,2.05,1.88,2.04,1.89,2.06,1.90,2.01,1.87,2.37,3.30,3.00,2.40,3.3,2.75,2.45,3.30,2.95,2.45,3.44,3.09,2.40,3.3,2.90,2.40,3.4,3.00,2.50,3.55,3.18,2.43,3.36,3.02,1.95,1.95,1.96,1.94,2.09,1.96,1.96,1.87,-0.25,2.08,1.85,2.10,1.84,2.14,1.87,2.08,1.81
4,6/8/2022,15:00,Newcastle,Nott'm Forest,H,S Hooper,0,0,1.66,3.8,5.25,1.65,3.80,5.50,1.65,3.80,5.50,1.71,3.74,5.83,1.67,3.7,5.25,1.62,3.7,5.50,1.72,3.96,6.00,1.67,3.80,5.57,2.05,1.85,2.10,1.81,2.10,1.86,2.03,1.81,-0.75,1.87,2.06,1.92,2.01,1.92,2.08,1.86,2.02,1.53,4.00,6.00,1.58,3.9,6.00,1.63,3.80,6.00,1.57,4.22,6.60,1.53,3.9,6.50,1.57,3.9,7.00,1.67,4.30,7.00,1.59,4.07,6.15,1.94,1.96,1.97,1.93,2.06,1.97,1.94,1.89,-1.00,1.97,1.96,1.99,1.93,2.19,1.97,2.03,1.86


In [97]:
features = ['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
          'PSH', 'PSD', 'PSA', 'WHH','WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH', 'AvgD', 'AvgA', 'B365>2.5',
          'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH','B365AHA', 'PAHH', 'PAHA', 
          'MaxAHH', 'MaxAHA', "AvgAHH", 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA',
          'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD',
          'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH',
          'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA']

X = StandardScaler().fit_transform(data_winner[features])
#X = OneHotEncoder(sparse_ouput = False, ['TeamH', 'TeamA', 'Referee']) 
            
y = data_winner1['FTR']

le = LabelEncoder().fit(y)
y = le.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 321)

In [98]:
#Start with a sequential model
model = tf.keras.Sequential()

# Then add dense layer with relu activation
model.add(tf.keras.layers.Dense(units = 32,
                                input_shape = (len(features),),
                                activation = 'relu'))
# And end with an output layer with softmax activation (since this is a multiclass problem)
model.add(tf.keras.layers.Dense(3, activation = 'softmax'))

# Finally, compile the model
model.compile('adam', 'sparse_categorical_crossentropy')

In [99]:
model.fit(X_train, y_train, epochs = 100)

Epoch 1/100
9/9 [==============================] - 0s 3ms/step - loss: 1.1418
Epoch 2/100
9/9 [==============================] - 0s 55us/step - loss: 1.0416
Epoch 3/100
9/9 [==============================] - 0s 2ms/step - loss: 1.0053
Epoch 4/100
9/9 [==============================] - 0s 1ms/step - loss: 0.9812
Epoch 5/100
9/9 [==============================] - 0s 1ms/step - loss: 0.9725
Epoch 6/100
9/9 [==============================] - 0s 624us/step - loss: 0.9551
Epoch 7/100
9/9 [==============================] - 0s 1ms/step - loss: 0.9502
Epoch 8/100
9/9 [==============================] - 0s 1ms/step - loss: 0.9445
Epoch 9/100
9/9 [==============================] - 0s 423us/step - loss: 0.9423
Epoch 10/100
9/9 [==============================] - 0s 840us/step - loss: 0.9348
Epoch 11/100
9/9 [==============================] - 0s 1ms/step - loss: 0.9333
Epoch 12/100
9/9 [==============================] - 0s 1ms/step - loss: 0.9325
Epoch 13/100
9/9 [==============================] - 0s

In [100]:
model.predict(X_test)

3/3 [==============================] - 0s 526us/step


array([[0.1547601 , 0.47015584, 0.37508386],
       [0.57767713, 0.18088225, 0.24144058],
       [0.47653404, 0.11325288, 0.41021308],
       [0.37196878, 0.18860564, 0.4394255 ],
       [0.24790329, 0.19728152, 0.5548151 ],
       [0.48453113, 0.2023193 , 0.31314948],
       [0.01978109, 0.46066275, 0.51955605],
       [0.36175716, 0.27401388, 0.364229  ],
       [0.6389775 , 0.17002515, 0.19099736],
       [0.4811937 , 0.32210782, 0.19669846],
       [0.7882274 , 0.11417334, 0.09759923],
       [0.5270603 , 0.19459796, 0.2783418 ],
       [0.21946672, 0.18156315, 0.59897006],
       [0.7005297 , 0.21832411, 0.08114616],
       [0.4056305 , 0.17286725, 0.42150226],
       [0.2851178 , 0.22723928, 0.48764285],
       [0.11679855, 0.32084975, 0.5623517 ],
       [0.7220478 , 0.19961251, 0.07833981],
       [0.13954493, 0.36838675, 0.49206826],
       [0.03434232, 0.14220741, 0.82345027],
       [0.43073404, 0.25939813, 0.30986774],
       [0.63172835, 0.11506267, 0.25320885],
       [0.

In [101]:
y_pred = np.argmax(model.predict(X_test), axis = 1)
y_pred

3/3 [==============================] - 0s 1ms/step


array([1, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 0, 0,
       0, 0, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 0, 2, 2, 2, 2, 1, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 2, 1, 2, 2,
       2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 0, 2, 0, 0, 2,
       2, 0, 1, 2, 0, 0, 1], dtype=int64)

In [102]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.44      0.56      0.49        27
           1       0.14      0.05      0.07        22
           2       0.61      0.72      0.66        46

    accuracy                           0.52        95
   macro avg       0.40      0.44      0.41        95
weighted avg       0.45      0.52      0.48        95



In [103]:
confusion_matrix(y_test, y_pred)

array([[15,  3,  9],
       [ 9,  1, 12],
       [10,  3, 33]], dtype=int64)